# Introduction to HydroGenerate

HydroGenerate is...

This document shows a quick workflow for HydroGenerate functions.

In [61]:
%load_ext autoreload
%autoreload 2
# Standard imports
import pandas as pd
import matplotlib.pyplot as plt
import urllib3
urllib3.disable_warnings()

# Local imports
from hat.api_call import *
from hat.utils.hydropower_potential import *
from hat.utils.PyGeoTools.geolocation import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Example 1. Basic Calculation:

This section presents the workflow for basic hydropower calculations. Basic calculations compute power, head, or flow, given any combination of two parameters. Basic calculations are included for users who do not have data about their system, and it is intended to provide a general estimation only. 

In [74]:
# 1.1) Calculate power from a given head and flow

flow = 8000 # given flow, in cfs
head = 20 # head, in ft
power = None

flow, head, hp = calculate_hp_potential(flow= flow, head= head, rated_power= power)
print("Hydropower potential is {} Kw".format(round(hp, 0)))


Hydropower potential is 11515.0 Kw


In [75]:
# 1.2) Calculate head from a given head and flow

flow = 8000 # given flow, in cfs
head = None # head, in ft
power = 11515

flow, head, hp = calculate_hp_potential(flow= flow, head= head, rated_power= power)
print("The head required to obtain {} Kw with a flow of {} cfs is {} ft".format(power, flow, round(head, 0)))

The head required to obtain 11515 Kw with a flow of 8000.0 cfs is 20.0 ft


In [76]:
# 1.3) Calculate head from a given head and flow

flow = None # given flow, in cfs
head = 20 # head, in ft
power = 11515

flow, head, hp = calculate_hp_potential(flow= flow, head= head, rated_power= power)
print("The flow required to obtain {} Kw with a head of {} ft is {} cfs".format(power, head, round(flow, 0)))

The flow required to obtain 11515 Kw with a head of 20.0 ft is 8000.0 cfs


### Example 2. Diversion:

A diversion, sometimes called a “run-of-river” facility, channels a portion of a river through a canal and/or a penstock to utilize the natural decline of the river bed elevation to produce energy. A penstock is a closed conduit that channels the flow of water to turbines with water flow regulated by gates, valves, and turbines. A diversion may not require the use of a dam. [DOE](https://www.energy.gov/eere/water/types-hydropower-plants#:~:text=There%20are%20three%20types%20of,renewable%20energy%20to%20the%20grid.)

In diversion projects, HydroGenerate focuses on calculating hydropwoer under diffrent configurations.

In [119]:
# 1.1) Head, power, and lenght of penstock are known. FLow is a single value.
# In this scenario HydroGenerate will select a turbine, compute effieincy for the given flow
# and values within 0.6 to 1.2 the given flow, penstock diameter (assuming steel if no material is given),
# head loss for all flows, rater power,
# power a given range of flow, 

flow = 8000
head = 20
power = None

hp = calculate_hp_potential(flow= flow, rated_power= power, head= head,
                           hydropower_type= 'Diversion', penstock_length = 50)

# Explore output
print('Head_loss at design flow (ft):', round(hp.head_loss, 2))
print('Flow range evaluated (cfs):', np.round(hp.flow, 1))
print('Design flow (cfs):', hp.design_flow)
print('Turbine type:',hp.turbine_type)
print('Turbine Efficiency for the given flow range:', np.round(hp.effi_cal,3))
print('Rated Power (Kw):',round(hp.rated_power,1))
print('Power (Kw) for diffrent flow ranges:', np.round(hp.power,1))
print('Generator Efficiency:',hp.generator_efficiency)
print('Head Loss method:',hp.headloss_method)
print('Net head (ft):', round(hp.net_head,1))
print('Penstock length (ft):', hp.penstock_length)
print('Penstock diameter (ft):', round(hp.penstock_diameter,2))

Head_loss at design flow (ft): 1.96
Flow range evaluated (cfs): [4800.  5082.4 5364.7 5647.1 5929.4 6211.8 6494.1 6776.5 7058.8 7341.2
 7623.5 7905.9 8188.2 8470.6 8752.9 9035.3 9317.6 9600. ]
Design flow (cfs): 8000.0
Turbine type: Kaplan
Turbine Efficiency for the given flow range: [0.911 0.911 0.911 0.911 0.911 0.911 0.911 0.911 0.911 0.911 0.91  0.908
 0.904 0.895 0.881 0.858 0.82  0.762]
Rated Power (Kw): 10906.9
Power (Kw) for diffrent flow ranges: [ 6542.7  6928.8  7314.   7699.   8083.9  8468.9  8853.8  9238.6  9622.7
 10004.3 10379.3 10739.8 11071.5 11351.4 11543.7 11595.7 11432.5 10951. ]
Generator Efficiency: 0.98
Head Loss method: Darcy-Weisbach
Net head (ft): 18.0
Penstock length (ft): 50.0
Penstock diameter (ft): 12.83


In [120]:
# Full list of parameters in cluded in HydroGenerate calculation
dir(hp)

['Re',
 'Rm',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'design_diameter',
 'design_flow',
 'designdiameter_calculator',
 'effi_cal',
 'flow',
 'generator_efficiency',
 'h',
 'head',
 'head_loss',
 'head_loss_calculation',
 'headloss_method',
 'max_headloss_allowed',
 'net_head',
 'pctime_runfull',
 'pelton_n_jets',
 'penstock_diameter',
 'penstock_frictionfactor',
 'penstock_length',
 'penstock_material',
 'pmax',
 'pmin',
 'power',
 'rated_power',
 'raw_power',
 'reynoldsnumber_calculator',
 'runner_diameter',
 'turb_cap',
 'turbine_type']

In [122]:
# 1.1) Exploring additional options:
#  Using Hazen-Williams for head loss calculation

flow = 8000
head = 20
power = None

hp = calculate_hp_potential(flow= flow, rated_power= power, head= head,
                           hydropower_type= 'Diversion', penstock_length = 50,
                           headloss_method= "Hazen-Williams")

# Explore output
print('Head_loss at design flow (ft):', round(hp.head_loss, 2))
print('Rated Power (Kw):',round(hp.rated_power,1))
print('Head Loss method:',hp.headloss_method)
print('Net head (ft):', round(hp.net_head,1))
print('Penstock diameter (ft):', round(hp.penstock_diameter,2))

Head_loss at design flow (ft): 2.0
Rated Power (Kw): 10885.6
Head Loss method: Hazen-Williams
Net head (ft): 18.0
Penstock diameter (ft): 12.24


In [166]:
# 1.1) Exploring additional options:
#  Using Hazen-Williams for head loss calculation and 
# using a known C

# 1.1) Exploring additional options:
#  Selecting a diffrent material for the penstock

flow = 8000
head = 20
power = None

hp = calculate_hp_potential(flow= flow, rated_power= power, head= head,
                           hydropower_type= 'Diversion', penstock_length = 50,
                           headloss_method= 'Hazen-Williams', penstock_material= 'Concrete')

# Explore output
print('Head_loss at design flow (ft):', round(hp.head_loss, 2))
print('Rated Power (Kw):',round(hp.rated_power,1))
print('Head Loss method:',hp.headloss_method)
print('Net head (ft):', round(hp.net_head,1))
print('Penstock diameter (ft):', round(hp.penstock_diameter,2))
print('Hazen-Williams C:', round(hp.penstock_frictionfactor,2))

Head_loss at design flow (ft): 2.0
Rated Power (Kw): 10885.6
Head Loss method: Hazen-Williams
Net head (ft): 18.0
Penstock diameter (ft): 12.76
Hazen-Williams C: 130


In [169]:
# 1.1) Exploring additional options:
#  Using Hazen-Williams for head loss calculation and
# using a diffrent C value
# Note: editing hydraulic_processing.py allows for entering materials that can be called by name


flow = 8000
head = 20
power = None

hp = calculate_hp_potential(flow= flow, rated_power= power, head= head,
                           hydropower_type= 'Diversion', penstock_length = 50,
                           headloss_method= 'Hazen-Williams', penstock_frictionfactor= 100)

# Explore output
print('Head_loss at design flow (ft):', round(hp.head_loss, 2))
print('Rated Power (Kw):',round(hp.rated_power,1))
print('Head Loss method:',hp.headloss_method)
print('Net head (ft):', round(hp.net_head,1))
print('Penstock diameter (ft):', round(hp.penstock_diameter,2))

Head_loss at design flow (ft): 2.0
Rated Power (Kw): 10885.6
Head Loss method: Hazen-Williams
Net head (ft): 18.0
Penstock diameter (ft): 14.1
